In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler


from vgg16_2 import VGG_16_2


import argparse
from data_loader_half_2 import get_loader
from data_loader_half_2 import get_dataset

import math

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='2'
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair']
celeba_crop_size = 224
image_size = 224 
batch_size = 30
num_workers = 0
mode1 = 'train' #test or train
mode2 = 'test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #"cuda:0" if torch.cuda.is_available() else 
torch.set_default_tensor_type('torch.cuda.FloatTensor')


 



In [3]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [4]:
celeba_loader_test2, data_test2 = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, 1,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...


# Validate

In [6]:
def validate(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()
        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)


In [7]:
def validate_per_example(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



In [8]:
def validate_per_example_mask(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        im = im
        preds, _ = model(im,mask)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



# Train

In [9]:
def train(model, criterion, optimizer, increase= False, coef=1):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        print("iteration no", i)
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        
        
        
        optimizer.zero_grad()
        output,noisy = model(im)
        
        
        print(torch.mean(model.intermed.scales()))
        print(torch.max(model.intermed.scales()))
        print(torch.min(model.intermed.scales()))
        
        if (increase):
            #loss = criterion(output, labels) + coef* 1/(torch.mean(model.intermed.scales()) )
            print(labels.shape)
            print(labels)
            loss = criterion(output, labels) - coef* torch.log(torch.mean(model.intermed.scales()) ) + 10
            
        else:
            loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()
        del im
        del labels
        del output
        del values
        del indices
        
        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
     
        





In [10]:
def train_mask(model, criterion, optimizer, mask, increase= False, coef=1):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        print("iteration no", i)
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        
        
        im = im 
        optimizer.zero_grad()
        output,noisy = model(im,mask)
        
        
        print(torch.mean(model.intermed.scales()))
        print(torch.max(model.intermed.scales()))
        print(torch.min(model.intermed.scales()))
        
        if (increase):
            print(labels.shape)
            print(labels)
            #loss = criterion(output, labels) + coef* 1/(torch.mean(model.intermed.scales()) )
            loss = criterion(output, labels) - coef* torch.log(torch.mean(model.intermed.scales()) ) + 10
        else:
            loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()
        del im
        del labels
        del output
        del values
        del indices
        
        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
     
        





# Noisy Network

In [11]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        #self.rhos.requires_grad = False
        #self.locs.requires_grad = True
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)*mask
        return  self.locs + self.scales()*epsilon
                                 
                               
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise



class vgg_syn(nn.Module):

    def __init__(self, model_features, model_classifier, min_scale,max_scale, given_locs, given_scale):
        super(vgg_syn, self).__init__()
        

        self.intermed = NoisyActivation( given_locs, given_scale, min_scale, max_scale)
        self.model_pt2 =  torch.nn.Sequential(*(list(model_features)))
        self.model_pt3 = model_classifier
        #self.components = components
        for child in itertools.chain(self.model_pt2, self.model_pt3): #self.model_pt2 #(self.model_pt2, 
            for param in child.parameters():
                param.requires_grad = False
            if isinstance(child, nn.modules.batchnorm._BatchNorm):
                child.eval()
                child.affine = False
                child.track_running_stats = False
         
                                 
    def forward(self, img, mask):
                                 
        img = img
        x = self.intermed(img, mask)
        noisy = x.detach()
       
        x = self.model_pt2(x)                    
        x = x.view(img.size(0), -1)
        x = self.model_pt3(x)                                 

        return x, noisy
    
                                 
 

In [12]:

model = VGG_16_2()

model.load_state_dict(torch.load("vgg-hair-less-84l88",  map_location=device))


criterion = nn.NLLLoss()
mus = torch.zeros((3,224,224))
scale = torch.ones((3,224,224))*0.001
model_syn = vgg_syn(model.convnet, model.fc ,0.0,5 ,mus, scale )
model_syn.load_state_dict(torch.load("celeba-vgg-hair-avg-cloak", map_location=device))


<All keys matched successfully>

In [13]:
#mult_mask =  torch.ones(model_syn.intermed.rhos.shape)
mult_mask = torch.where(model_syn.intermed.scales()>4.5, torch.zeros(model_syn.intermed.scales().shape), torch.ones(model_syn.intermed.scales().shape))

In [14]:
criterion = nn.CrossEntropyLoss().to(device)

In [15]:
d = list(filter(lambda p: p.requires_grad, model_syn.parameters()))
print(len(d))

2


In [55]:
lr =  0.001  #

for epoch in range (1): # real
   
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_syn.parameters()), lr=lr, weight_decay=0)
    cor, size = train_mask(model_syn, criterion, optimizer,mult_mask, True, 0.1)
    print (cor,size)
 
    validate_single_label(model_syn)
    


iteration no 0
tensor(4.8030, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1])
tensor(9.9007, grad_fn=<AddBackward0>) loss
0.7666666666666667
iteration no 1
tensor(4.8030, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 0])
tensor(9.9321, grad_fn=<AddBackward0>) loss
0.7833333333333333
iteration no 2
tensor(4.8030, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0])
tensor(9.9499, grad_fn=<AddBackward0>) loss
0.7666666666666667
iteration no 3
tensor(4.8030, grad_fn=<M

iteration no 26
tensor(4.8028, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0])
tensor(9.9732, grad_fn=<AddBackward0>) loss
0.7888888888888889
iteration no 27
tensor(4.8028, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 0, 0, 0])
tensor(10.0378, grad_fn=<AddBackward0>) loss
0.7857142857142857
iteration no 28
tensor(4.8028, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0])
tensor(10.0586, grad_fn=<AddBackward0>) loss
0.7816091954022989
iteration no 29
tensor(4.8028, grad

iteration no 52
tensor(4.8026, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 1, 0, 0])
tensor(9.8800, grad_fn=<AddBackward0>) loss
0.7710691823899372
iteration no 53
tensor(4.8026, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0])
tensor(9.9486, grad_fn=<AddBackward0>) loss
0.7709876543209877
iteration no 54
tensor(4.8026, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1])
tensor(9.9632, grad_fn=<AddBackward0>) loss
0.7703030303030303
iteration no 55
tensor(4.8026, grad_f

iteration no 78
tensor(4.8025, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 1])
tensor(10.0672, grad_fn=<AddBackward0>) loss
0.7746835443037975
iteration no 79
tensor(4.8025, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 0])
tensor(10.0377, grad_fn=<AddBackward0>) loss
0.77375
iteration no 80
tensor(4.8025, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0])
tensor(9.8591, grad_fn=<AddBackward0>) loss
0.7744855967078189
iteration no 81
tensor(4.8025, grad_fn=<MeanBa

iteration no 104
tensor(4.8024, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1])
tensor(9.8981, grad_fn=<AddBackward0>) loss
0.7746031746031746
iteration no 105
tensor(4.8024, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 0])
tensor(9.9816, grad_fn=<AddBackward0>) loss
0.7738993710691824
iteration no 106
tensor(4.8024, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0])
tensor(9.8799, grad_fn=<AddBackward0>) loss
0.7744548286604361
iteration no 107
tensor(4.8024, gr

...

tensor(9.9620, grad_fn=<AddBackward0>) loss
0.7709498834498835
iteration no 1144
tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1])
tensor(9.9832, grad_fn=<AddBackward0>) loss
0.7708879184861718
iteration no 1145
tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1])
tensor(10.0308, grad_fn=<AddBackward0>) loss
0.770826061663758
iteration no 1146
tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0])
tensor(9.9303, grad_fn=<AddBack

tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 0])
tensor(9.9771, grad_fn=<AddBackward0>) loss
0.7703387418161116
iteration no 1171
tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 0, 1])
tensor(9.9658, grad_fn=<AddBackward0>) loss
0.770278725824801
iteration no 1172
tensor(4.7989, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0])
tensor(9.9772, grad_fn=<AddBackward0>) loss
0.7702472293265132
iteration no 1173
tensor(4.7989, grad_fn=<MeanBack

tensor(9.7918, grad_fn=<AddBackward0>) loss
0.770819397993311
iteration no 1196
tensor(4.7988, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 0])
tensor(10.1279, grad_fn=<AddBackward0>) loss
0.7706766917293233
iteration no 1197
tensor(4.7988, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 1, 0, 0])
tensor(9.9655, grad_fn=<AddBackward0>) loss
0.7706455203116305
iteration no 1198
tensor(4.7988, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
tensor(9.8380, grad_fn=<AddBack

tensor(4.7986, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0])
tensor(9.9064, grad_fn=<AddBackward0>) loss
0.7710820387026438
iteration no 1223
tensor(4.7986, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0])
tensor(10.0231, grad_fn=<AddBackward0>) loss
0.7710511982570806
iteration no 1224
tensor(4.7986, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(0.0003, grad_fn=<MinBackward1>)
torch.Size([30])
tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1])
tensor(9.9334, grad_fn=<AddBackward0>) loss
0.7711020408163265
iteration no 1225
tensor(4.7986, grad_fn=<MeanBa

KeyboardInterrupt: 

In [16]:
validate_per_example_mask(model_syn, mult_mask) 

1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.875
0.7777777777777778
0.7
0.7272727272727273
0.75
0.7692307692307693
0.7857142857142857
0.8
0.8125
0.8235294117647058
0.7777777777777778
0.7894736842105263
0.8
0.8095238095238095
0.8181818181818182
0.8260869565217391
0.8333333333333334
0.84
0.8461538461538461
0.8518518518518519
0.8571428571428571
0.8620689655172413
0.8666666666666667
0.8709677419354839
0.875
0.8787878787878788
0.8823529411764706
0.8857142857142857
0.8888888888888888
0.8918918918918919
0.8947368421052632
0.8974358974358975
0.9
0.9024390243902439
0.9047619047619048
0.8837209302325582
0.8863636363636364
0.8888888888888888
0.8913043478260869
0.8936170212765957
0.8958333333333334
0.8979591836734694
0.88
0.8823529411764706
0.8846153846153846
0.8867924528301887
0.8888888888888888
0.8727272727272727
0.8571428571428571
0.8596491228070176
0.8620689655172413
0.847457627118644
0.85
0.8524590163934426
0.8548387096774194
0.8571428571428571
0.859375
0.8615384615384616
0.8636363636363636
0.86567164179104

0.7463312368972747
0.7468619246861925
0.7453027139874739
0.7458333333333333
0.7463617463617463
0.7448132780082988
0.7453416149068323
0.7458677685950413
0.7463917525773196
0.7469135802469136
0.7453798767967146
0.7459016393442623
0.7464212678936605
0.746938775510204
0.7474541751527495
0.7479674796747967
0.7484787018255578
0.7489878542510121
0.7474747474747475
0.7479838709677419
0.7484909456740443
0.748995983935743
0.7474949899799599
0.746
0.7465069860279441
0.7470119521912351
0.7475149105367793
0.748015873015873
0.7465346534653465
0.7450592885375494
0.7455621301775148
0.7460629921259843
0.7465618860510805
0.7470588235294118
0.7475538160469667
0.748046875
0.7485380116959064
0.7490272373540856
0.7495145631067961
0.75
0.7504835589941973
0.750965250965251
0.7514450867052023
0.7519230769230769
0.7504798464491362
0.7490421455938697
0.7476099426386233
0.7480916030534351
0.7485714285714286
0.747148288973384
0.7476280834914611
0.7462121212121212
0.7466918714555766
0.7471698113207547
0.74764595103

0.7586580086580087
0.7589189189189189
0.7591792656587473
0.7583603020496225
0.7586206896551724
0.7578040904198062
0.7580645161290323
0.7583243823845328
0.7585836909871244
0.7588424437299035
0.7580299785867237
0.7572192513368984
0.7564102564102564
0.7566702241195304
0.7569296375266524
0.7561235356762513
0.7553191489361702
0.7555791710945803
0.7558386411889597
0.7560975609756098
0.7563559322033898
0.7555555555555555
0.7547568710359408
0.7550158394931362
0.7542194092827004
0.7534246575342466
0.7536842105263157
0.7539432176656151
0.7542016806722689
0.7534102833158447
0.7536687631027253
0.7539267015706806
0.75418410041841
0.754440961337513
0.7546972860125261
0.7549530761209593
0.7552083333333334
0.7554630593132154
0.7546777546777547
0.754932502596054
0.7551867219917012
0.755440414507772
0.7556935817805382
0.7559462254395036
0.756198347107438
0.7564499484004128
0.756701030927835
0.756951596292482
0.757201646090535
0.7564234326824255
0.75564681724846
0.7558974358974359
0.7561475409836066
0.75

0.7631195335276968
0.763292061179898
0.7634643377001455
0.7629090909090909
0.7630813953488372
0.7632534495279594
0.7634252539912917
0.7635968092820885
0.763768115942029
0.7639391745112237
0.76410998552822
0.7642805495300072
0.7644508670520231
0.7646209386281588
0.7647907647907648
0.7642393655371305
0.7644092219020173
0.7638588912886969
0.7633093525179856
0.7634795111430626
0.7636494252873564
0.7638190954773869
0.7639885222381636
0.7634408602150538
0.7628939828080229
0.7630637079455977
0.7632331902718169
0.7626876340243031
0.7628571428571429
0.7630264097073519
0.7624821683309557
0.7626514611546685
0.7621082621082621
0.7615658362989324
0.7617354196301565
0.7619047619047619
0.7620738636363636
0.7622427253371186
0.7624113475177305
0.7625797306874557
0.7627478753541076
0.7629157820240623
0.7630834512022631
0.7632508833922261
0.7634180790960452
0.7635850388143967
0.7637517630465445
0.7639182522903453
0.7640845070422535
0.7642505277973258
0.7644163150492265
0.7645818692902319
0.76474719101123

0.7565934065934066
0.756727073036793
0.756860592755214
0.7569939659901261
0.7571271929824561
0.7572602739726028
0.7568455640744798
0.7569786535303776
0.7571115973741794
0.7572443958447239
0.7573770491803279
0.7575095576187876
0.75764192139738
0.7577741407528642
0.757360959651036
0.7574931880108992
0.7576252723311547
0.7577572128470332
0.7578890097932536
0.758020663404024
0.7581521739130435
0.7582835415535035
0.758414766558089
0.7585458491589799
0.7586767895878525
0.7588075880758808
0.7589382448537378
0.758527341635084
0.7581168831168831
0.7577068685776095
0.7578378378378379
0.7579686655861696
0.7580993520518359
0.7582298974635726
0.7583603020496225
0.7584905660377359
0.7580818965517241
0.7582121701669359
0.7583423035522067
0.7584722969338354
0.7586021505376344
0.7587318645889307
0.7588614393125671
0.7584541062801933
0.7580472103004292
0.7581769436997319
0.7583065380493034
0.7584359935725763
0.7585653104925053
0.7581594435527019
0.758288770053476
0.757883484767504
0.7580128205128205
0.7

...

0.7600286532951289
0.760050138776972
0.7599820948970457
0.7600035807000268
0.7599355531686359
0.7598675378143739
0.7598890281009486
0.759910514541387
0.7599319971367215
0.7598640064418001
0.75979602791197
0.7598175149834511
0.7598389982110912
0.7597710401574099
0.7597925236988017
0.7598140033980149
0.7598354792560801
0.7598569512740277
0.7597890219917754
0.7597211048538483
0.7597425813371469
0.7597640539815891
0.7596961572832887
0.7597176302385846
0.7597390993566834
0.7597605646386134
0.7597820260854029
0.7598034836980795
0.7598249374776705
0.7597570777886934
0.759778531880693
0.7597999821412627
0.7598214285714285
0.7597535934291582
0.7596857703981432
0.7597072212800143
0.7597286683327383
0.7597501115573405
0.7597715509548456
0.7597929865262782
0.7597251962883654
0.7597466321705773
0.759678858162355
0.7597002943537597
0.7596325365679629
0.7596539730669758
0.7596754057428214
0.7596968345965225
0.7597182596291013
0.7597396808415797
0.7597610982349795
0.7597825118103217
0.7598039215686274

0.7609387088455256
0.760959257349149
0.7609798023205845
0.7609144035751118
0.7609349488699837
0.7609554906341296
0.7608901108342642
0.7609106529209622
0.7609311914783953
0.7609517265074729
0.7609722580091042
0.7609927859841978
0.7610133104336625
0.7610338313584063
0.7610543487593372
0.7610748626373627
0.76109537299339
0.7611158798283262
0.7610505536005493
0.761071060762101
0.7610057495923797
0.7610262570791144
0.7610467610467611
0.7610672614962252
0.7610877584284121
0.7611082518442271
0.761128741744575
0.7611492281303602
0.7611697110024869
0.761190190361859
0.76121066620938
0.7612311385459534
0.7612516073724818
0.761272072689868
0.7612925344990144
0.7612272883099075
0.76124775044991
0.7612682090831191
0.7612886642104362
0.7613091158327622
0.761329563950998
0.7613500085660442
0.7613704496788009
0.7613908872901679
0.7614113214010448
0.7614317520123309
0.7613665553557668
0.7613869863013699
0.7613218046400136
0.76125663413799
0.7611914747924334
0.7612119137281753
0.7611467693624304
0.76108

0.7601521793069225
0.7600893152497519
0.7601091540560655
0.7601289895816107
0.7601488218272013
0.7601686507936508
0.7601884764817723
0.760208298892379
0.7601454665674849
0.7601652892561983
0.760185108668705
0.7602049248058173
0.7602247376683466
0.7601619299405156
0.7601817430813713
0.760201552948951
0.7601387626992648
0.7601585728444004
0.7600957965149888
0.7601156069364162
0.7601354140863678
0.7600726552179656
0.7600924626434409
0.7601122667987452
0.7600495253817582
0.7600693298118191
0.7600891309730131
0.7601089288661496
0.7601287234920373
0.7601485148514852
0.7601683029453016
0.7601880877742947
0.7602078693392724
0.7602276476410426
0.7601649484536083
0.760184727032822
0.7602045023501278
0.7602242744063324
0.7602440432022426
0.7602638087386645
0.7602835710164042
0.7603033300362677
0.7603230857990604
0.7602604252513598
0.7602801812937783
0.7602175346077785
0.7602372909285655
0.7602570439940682
0.7602767938050911
0.7602965403624382
0.7602339181286549
0.7602536649645857
0.76027340854813

0.759406887755102
0.7594260661618174
0.7594452415112386
0.7594644138040966
0.7594038890659867
0.7593433739740219
0.7592828685258964
0.7593020476456059
0.759321223709369
0.7592607344857802
0.759279910785407
0.7592990840302668
0.7593182542210896
0.7592577845026678
0.7592769549291288
0.759216498128832
0.759156050955414
0.7591752249024759
0.7591147906384333
0.7591339648173207
0.7591531359439669
0.7591723040191006
0.7591914690434506
0.7591310575316305
0.7591502227880331
0.7591693849948286
0.7591885441527446
0.7591281520960942
0.7590677696468342
0.7590073968026724
0.7590265627485288
0.7589662027833002
0.758985368956743
0.7590045320823726
0.7590236921609159
0.7590428491930996
0.7590620031796502
0.7590811541212941
0.759100302018757
0.7590399745688627
0.7590591226954864
0.7590782677791021
0.7590179564595583
0.7590371017716692
0.7590562440419447
0.7590753832711097
0.7590945194598888
0.7591136526090064
0.7591327827191868
0.7591519097911538
0.7591710338256312
0.7591107582373958
0.7591298825023817


(12700, tensor(9643))